<a href="https://colab.research.google.com/github/untreated-outlier/object-tracking-/blob/main/object_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Object tracking

env setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

In [2]:
!cp drive/MyDrive/object-tracking-main/tracker.py .

In [3]:
!cp drive/MyDrive/object-tracking-main/highway.mp4 .

 *Run from here*

In [4]:
import cv2
from tracker import *

In [5]:
#create tracker object
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture("highway.mp4")

#object detection from stable camera
object_detector = cv2.createBackgroundSubtractorMOG2( history = 100, varThreshold = 40 )

#extracts moving objects fom a stable camera
# from google.colab.patches import cv2_imshow

# while (cap.isOpened()):
#   success, frame = cap.read()
#   if success:
#     cv2_imshow(frame)

# cap.release()
# cv2.destroyAllWindows() 


In [ ]:
from google.colab.patches import cv2_imshow

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[340: 720,500: 800]

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2_imshow(roi)
    cv2_imshow(frame)
    cv2_imshow(mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()